# [Tensorflow - Help Protect the Great Barrier Reef](https://www.kaggle.com/c/tensorflow-great-barrier-reef)
> Detect crown-of-thorns starfish in underwater image data

## Notebooks:
- Train: [object-detection-by-cys-train](https://www.kaggle.com/code/cuiyushuai/object-detection-by-cys-train)

- Infer: [object-detection-by-cys-infer](https://www.kaggle.com/code/cuiyushuai/object-detection-by-cys-infer)

<br><br>
# Install Libraries
> 安装库

- -q选项用于将安装过程设为静默模式
- -U选项用于确保安装最新版本的包
- Loguru 是一个 Python 日志库，它提供了简单且强大的日志记录功能，使开发者能够更轻松地管理和输出日志信息。
- bbox-utility 是一个用于边界框操作的 Python 库
    - coco.py - 主要用于将不同格式的标注数据转换为 COCO 数据集格式的 JSON 文件
    - utils.py - 提供了一组用于处理图像边界框和标注数据的函数和类，适用于不同的边界框表示格式和计算机视觉任务。

In [ ]:
# bbox-utility, check https://github.com/cys02/bbox for source code
!pip install -q /kaggle/input/loguru-lib-dataset/loguru-0.5.3-py3-none-any.whl
!pip install -q /kaggle/input/bbox-lib-dataset

<br><br>
# Import Libraries
> 导入库

- `numpy`: 用于数值计算的基础库

- `tqdm`: 提供可视化的进度跟踪（在循环中显示进度条）

- `pandas`: 用于数据处理和分析的强大工具

- `os`: 提供访问操作系统功能的接口

- `cv2`: 导入 `OpenCV` 库，它是一个用于CV任务的开源库，提供图像和视频处理功能

- `matplotlib`: 用于绘图和数据可视化的库

- `glob`: 提供了查找文件路径的功能，可以根据通配符匹配模式获取文件列表

- `shutil`: 提供了一些高级的文件和目录操作函数，比如复制、移动和删除等

- `sys.path.append`: 将路径添加到Python解释器的搜索路径中，为了导入指定路径下的库

- `torch`: 导入 `torch` 框架

- `PIL`: 用于图像处理的Python库，提供了打开、操作和保存各种图像格式等功能

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image

<br><br>
# Key-Points
> 关键点

* 与以往的目标检测比赛有所不同，参赛者必须使用**python时序API**提交预测
* 每行预测都需要包含图像的所有边界框。提交格式采用**COCO**格式，即 `[x_min、y_min、width、height]`
* 比赛的评价指标采用F2指标，它容忍一些假正例（FP），以确保较少海星被错过。这意味着解决假反例（FN）比假正例（FP）更重要
$$F2 = 5 \cdot \frac{precision \cdot recall}{4\cdot precision + recall}$$

<br><br>
# Meta Data
> 元数据，对数据集中元素的介绍

* `train_images/` - 包含训练集照片的文件夹，照片的命名形式为 `video_{video_id}/{video_frame}.jpg`.
* `[train/test].csv` - 图像的元数据。与其他测试文件一样，在提交笔记本之前，大部分测试元数据只能在笔记本中使用，只有前几行可供下载
* `video_id` - 图像所属视频的ID编号。数据集中的图片来源于三个视频的抽帧，这个参数表明图片来自于哪个视频
* `video_frame` - 视频中图像的帧号。从潜水员浮出水面开始，帧号中偶尔会出现空缺
* `sequence` - 给出视频中无间隙子集的 ID。序列 ID 并非有序排列
* `sequence_frame` - 给出序列中的帧号。
* `image_id` - 图像的 ID 代码，格式为 `{video_id}-{video_frame}`
* `annotations` - 检测到的海星的的边界框，可直接用 Python 评估的字符串格式表示。与将、提交的预测使用的格式不同。在 test.csv 中不可用。边界框由图像左下角的像素坐标"(x_min, y_min) "及其像素 "宽 "和 "高 "来描述（COCO 格式）。
 
 
 在中文语境下，“annotations”直译为“注释”，所以后文中可能用“**注释**”、“**边界框**”或“**标注框**”来表示图片中**对海星的选择框**
 
 *  `CKPT_PATH` - 指定模型检查点的路径，导入训练好的模型 `best.pt` 
 *  `CONF` - 置信度阈值（Confidence Threshold）。在目标检测任务中，模型会预测每个检测框中物体存在的置信度。只有当置信度高于这个阈值时，检测结果才会被保留。
 *  `IOU` - 交并比阈值。IOU用于判断两个检测框的重叠程度，如果两个框的IOU大于这个阈值，它们可能会被认为是同一个目标。

In [ ]:
# 指定根目录路径
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# 指定模型的检查点路径（训练好的模型best.pt）
CKPT_PATH = '/kaggle/input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt'
# 定义图像的大小
IMG_SIZE  = 9000
# 定义置信度阈值
CONF      = 0.25
# 定义了IOU（交并比）阈值
IOU       = 0.40
# 定义了是否进行数据增强
AUGMENT   = True

In [ ]:
# 读取训练数据
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
# 在数据df中添加image_path列，列值为图片路径
df['image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
# 在数据df中添加annotations列，列值为标注框坐标数据
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))

## Number of BBoxes
> 查看多少图片有标注框

通过前面对数据集的查看，我们发现前两个数据的 `annotations` 都为空，也就是没有标注框，下面我们查看一下有多少数据有标注

In [ ]:
# 使用progress_apply函数将lambda函数应用于df['annotations']列的每个元素
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
# 计算有多少图像没有边界框，有多少图像有边界框，并将数值转换为百分比
data = (df.num_bbox>0).value_counts(normalize=True)*100
# 输出
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

<br><br>
# Helper
> 辅助工具

* `coco2yolo`, `coco2voc`, `voc2yolo` - 用于在不同的标注格式之间进行转换
* `draw_bboxes` - 用于在图像上绘制边界框
* `load_image` - 用于加载图像
* `clip_bbox` - 用于裁剪边界框，使其适应图像的边界
* `str2annot` - 用于将字符串格式的边界框注释转换为字典格式
* `annot2str` - 用于将字典格式的边界框注释转换为字符串格式

In [ ]:
from bbox.utils import coco2yolo, coco2voc, voc2yolo, voc2coco
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

# 参数annots表示边界框的注释列表
def get_bbox(annots):
    # 注释字典的值转换为列表，并将结果存储在bboxes变量中
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

# 参数row表示数据集中的一行数据
def get_imgsize(row):
    # 函数用imagesize.get函数获取该图像的宽度和高度，宽度和高度被添加到row字典中
    row['width'], row['height'] = imagesize.get(row['image_path'])
    # 返回更新后的row字典
    return row

# 设定随机数生成器的种子为32，以确保每次运行代码时产生相同的随机数序列
np.random.seed(32)

# 这一行代码使用列表推导式生成一个包含一个随机颜色元组的列表
# 列表中只有一个元素，因为循环的范围是range(1)，这样就只生成了一个随机颜色
# 这些随机颜色可以用于在图像上绘制边界框时进行标记或区分不同的边界框
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

<br><br>
# [YOLOv5](https://github.com/ultralytics/yolov5/)
> 导入YOLOv5

- 在第一段代码中，我们创建了相应的配置文件夹，并将字体数据复制到文件夹下
- 在第二段代码中，我们定义了 `load_model` 函数，用于加载YOLOv5模型

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
# load_model函数，用于加载YOLOv5模型
# 参数：模型的路径，置信度阈值，IoU 阈值
def load_model(ckpt_path, conf=0.25, iou=0.50):
    # 从输出数据中加载yolov5模型
    # custom 表示加载一个自定义的模型
    # path=ckpt_path 指定了检查点的路径（训练好的模型）
    # source='local' 表示从本地加载模型
    # force_reload=True 强制重新加载模型
    model = torch.hub.load('/kaggle/input/yolov5-lib-dataset',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)
    # 置信度阈值
    model.conf = conf
    # 交并比阈值
    model.iou  = iou
    # 不对特定类别进行过滤，即所有类别都会被考虑
    model.classes = None
    # 表示每个框只能对应一个标签
    model.multi_label = False
    # 表示每张图像的最大检测数为 1000
    model.max_det = 1000
    return model

<br><br>
# Inference
> 推理

## Helper
> 辅助推理的方法

-  `predict` - 使用给定的模型对输入图像进行预测，并返回检测到的边界框和对应的置信度
-  `format_prediction` - 用于将置信度和边界框格式化为一串字符串注释
-  `show_img` - 用于在图像上绘制边界框并返回可视化结果

In [ ]:
# predict函数使用给定的模型对输入图像进行预测，并返回检测到的边界框和对应的置信度
# model: 预测使用的目标检测模型
# img: 输入图像
# size: 自定义推理大小，默认为768
# augment: 是否进行数据增强，默认为False
def predict(model, img, size=768, augment=False):
    # 获取高度和宽度
    height, width = img.shape[:2]
    # 使用模型对输入图像进行预测，不进行数据增强
    results = model(img, size=size, augment=augment)
    # 将预测结果转换为 pandas 数据帧，并提取其中的边界框坐标信息
    preds   = results.pandas().xyxy[0]
    # 从预测结果中提取边界框的 xmin、ymin、xmax 和 ymax 坐标，保存在一个 numpy 数组中
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    # 如果存在边界框结果
    if len(bboxes):
        # 将边界框坐标从 VOC 格式转换为 COCO 格式，并将结果转换为整数类型
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        # 提取预测结果中的置信度值
        confs   = preds.confidence.values
        # 返回边界框和置信度
        return bboxes, confs
    else:
        return [],[]

# format_prediction函数用于将置信度和边界框格式化为一串字符串注释
# 参数包含边界框的位置和置信度
def format_prediction(bboxes, confs):
    # 初始化注释字符串
    annot = ''
    # 如果存在边界框
    if len(bboxes)>0:
        # 使用循环遍历每个边界框
        for idx in range(len(bboxes)):
            # 从边界框坐标中提取 xmin、ymin、宽度 w 和高度 h
            xmin, ymin, w, h = bboxes[idx]
            # 提取对应的置信度
            conf             = confs[idx]
            # 将置信度和边界框坐标格式化为字符串，按一定格式连接起来
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            # 在每个边界框的字符串之间添加空格
            annot +=' '
        # 去除字符串末尾的多余空格
        annot = annot.strip(' ')
    # 返回格式化后的字符串注释
    return annot

# show_img用于在图像上绘制边界框并返回可视化结果
def show_img(img, bboxes, bbox_format='yolo'):
    # 创建一个列表 names，其中包含 'starfish' 字符串，长度与边界框数目相同
    names  = ['starfish']*len(bboxes)
    # 创建一个列表 labels，其中包含全为 0 的整数，长度与边界框数目相同
    labels = [0]*len(bboxes)
    # 使用外部的 draw_bboxes 函数，在图像上绘制边界框
    img    = draw_bboxes(img = img,                     # 输入的图像，表示要在其上绘制边界框
                           bboxes = bboxes,             # 边界框坐标的列表或数组，表示要绘制的边界框
                           classes = names,             # 类别名称的列表，表示每个边界框对应的类别名称
                           class_ids = labels,          # 类别标签的列表，表示每个边界框对应的类别标签
                           class_name = True,           # 一个布尔值，表示是否在边界框上显示类别名称
                           colors = colors,             # 边界框的颜色，表示每个类别对应的颜色
                           bbox_format = bbox_format,   # 边界框坐标格式，这里是'yolo'
                           line_thickness = 2)          # 绘制边界框时的线条粗细
    # 将绘制后的图像转换为 PIL 图像，并调整大小为 (800, 400)，最终返回可视化的 PIL 图像
    return Image.fromarray(img).resize((800, 400))

## Run Inference on **Train**
> 在训练集上运行推理，可视化推理结果

In [ ]:
# 调用函数加载模型
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# 从 DataFrame df 中选择具有大于1个边界框的图像路径，随机选择其中100个，并将它们存储在名为 image_paths 的列表中。
image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
# 遍历 image_paths 列表中的图像路径，使用索引 idx 和路径 path。
for idx, path in enumerate(image_paths):
    # 使用 OpenCV 加载图像，然后通过切片操作将通道从 BGR 转换为 RGB
    img = cv2.imread(path)[...,::-1]
    # 使用预加载的模型 model 对当前图像 img 进行目标检测，返回检测到的边界框坐标和置信度
    bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    # 使用 show_img 函数显示带有绘制的边界框的图像。
    # 图像、边界框坐标和边界框坐标格式（'coco'）被传递给 show_img 函数，然后使用 display 函数显示图像。
    display(show_img(img, bboxes, bbox_format='coco'))
    # 仅显示部分图像
    if idx>5:
        break

## Init Environment
> 初始化环境，比赛要求必须使用提供的 python 时序 API 提交运行结果

In [ ]:
import greatbarrierreef
# 初始化环境
env = greatbarrierreef.make_env()   # initialize the environment
# 迭代器，它循环遍历测试集和提交的样本
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

## Run Inference on **Test**
> 将测试集带入模型进行预测

In [ ]:
# 加载模型
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# 遍历迭代器 iter_test 中的测试图像和相关数据
# img, pred_df 是从迭代器中解包得到的当前测试图像和相关数据
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    # 使用预加载的模型 model 对当前图像 img 进行目标检测，返回检测到的边界框坐标和置信度
    bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    # 使用 format_prediction 函数将边界框坐标和置信度格式化为一串字符串的注释
    annot          = format_prediction(bboxes, confs)
    # 将格式化的注释赋值给测试数据中的 annotations 列
    pred_df['annotations'] = annot
    # 调用 env 的 predict 方法，将包含注释的预测数据传递给它。
    env.predict(pred_df)
    # 输出前三张图像的检测结果
    if idx<3:
        display(show_img(img, bboxes, bbox_format='coco'))

<br><br>
# Check Submission
> 查看提交数据

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()